# Analizando datos de Enfermedades Transmitidas por Vector (Dengue) en México para los años 2021 y 2023

### Importando librerías

In [19]:
import sqlite3
import pandas as pd
import numpy as np
from unidecode import unidecode
import os

### Conexión a la BD

In [20]:
# Se crea la conexión a la BD
connection = sqlite3.connect("dengue.db")

# Comprobar que la base de datos se ha creado correctamente.
print (connection.total_changes)

# Crear el cursor que te va a permitir enviar instrucciones en SQL.
cursor = connection.cursor()

0


Probemos la conexión ejecutando una consulta que nos regrese el número de registros que tiene la tabla:

In [21]:
cursor.execute("SELECT count(*) FROM dengue_301221")
rows = cursor.fetchall()

print(rows)

[(35413,)]


Primero, creamos una vista que contenga todos los registros donde la Entidad de Residencia sea Sonora:

In [22]:
# Se crea vista
"""
cursor.execute("CREATE VIEW vw_dengue_sonora AS SELECT * FROM dengue_301221 WHERE ENTIDAD_RES = 26")
rows = cursor.fetchall()
print(rows)
""";

Ahora podemos realizar consultas sobre la vista:


In [23]:
df_dengue_sonora = pd.read_sql_query("SELECT * FROM vw_dengue_sonora", connection)
df_dengue_sonora

,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
0,2021-12-13,913179,2,15,26,66,2,2,26,66,...,2,2,2,2,5,1,5,3,26,66
1,2021-12-13,920550,2,50,26,30,2,2,26,30,...,2,2,2,2,5,2,5,1,26,30
2,2021-12-13,910208,1,52,26,18,2,2,26,18,...,2,2,2,1,4,1,5,1,26,18
3,2021-12-13,920833,2,13,26,18,2,2,26,18,...,2,2,2,2,5,2,5,1,26,18
4,2021-12-13,911975,2,38,26,30,2,2,26,30,...,2,2,2,2,5,1,5,3,26,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,2021-12-13,923505,1,52,26,30,2,2,26,30,...,2,2,2,2,5,1,5,3,26,30
433,2021-12-13,923800,2,24,26,3,2,2,26,42,...,2,2,2,2,5,1,5,2,26,3
434,2021-12-13,894915,2,44,26,29,2,2,26,29,...,2,2,2,2,5,1,5,3,26,29
435,2021-12-13,901370,2,25,26,29,2,2,26,29,...,2,2,2,2,5,1,5,3,26,29


Se crean los diccionarios para cargar los catálogos... 

In [48]:
# Nombre del archivo Excel
excel_file = "diccionario_datos\Catálogos_Dengue.xlsx"

# Cargar el archivo Excel
excel_data = pd.ExcelFile(excel_file)

# Obtener el nombre de todas las hojas en el archivo Excel
sheet_names = excel_data.sheet_names

# Iterar sobre cada hoja y guardarla como archivo CSV
for sheet_name in sheet_names:
    # Leer los datos de la hoja actual
    df = excel_data.parse(sheet_name)
    
    # Quitar los acentos y remplazar los espacios en los nombres...
    sheet_name = '_'.join(sheet_name.split()).upper()
    sheet_name = unidecode(sheet_name)

    # Crear el nombre del archivo CSV (usando el nombre de la hoja)
    csv_file = f'diccionario_datos\Catálogos\{sheet_name}.csv'
    
    # Guardar los datos en el archivo CSV
    df.to_csv(csv_file, index=False)
    
    print(f'Guardando {csv_file}')

# Cerrar el archivo Excel
excel_data.close()

# Luego se cargan a un diccionario de Python que contendrá todos los catálogos...
# Ruta de la carpeta que contiene los Catálogos... CSV
carpeta = 'diccionario_datos\Catálogos'

# Diccionario para almacenar los diccionarios de cada archivo CSV
diccionarios = {}

# Iterar sobre cada archivo en la carpeta
for archivo in os.listdir(carpeta):
    if archivo.endswith('.csv'):
        # Crear la ruta completa al archivo
        ruta_completa = os.path.join(carpeta, archivo)
        
        # Cargar el archivo CSV en un DataFrame de pandas
        df = pd.read_csv(ruta_completa)
        
        # Crear un diccionario con el nombre del archivo como clave
        # y la primera columna como clave y la segunda columna como valor
        diccionario = dict(zip(df.iloc[:, 0], df.iloc[:, 1]))
        
        # Agregar el diccionario al diccionario principal, y de paso quitarle la extensión al nombre (.csv)...
        diccionarios[archivo[:-4]] = diccionario

# Ver los diccionarios...
diccionarios.keys()

# Ahora, diccionarios contiene un diccionario para cada archivo CSV con la primera columna como clave y la segunda columna como valor...

Guardando diccionario_datos\Catálogos\CATALOGO_ESTATUS_CASO.csv
Guardando diccionario_datos\Catálogos\CATALOGO_RESULTADO_PCR.csv
Guardando diccionario_datos\Catálogos\CATALOGO_DICTAMEN.csv
Guardando diccionario_datos\Catálogos\CATALOGO_INSTITUCION.csv
Guardando diccionario_datos\Catálogos\CATALOGO_TIPO_PACIENTE.csv
Guardando diccionario_datos\Catálogos\CATALOGO_MUNICIPIO.csv
Guardando diccionario_datos\Catálogos\CATALOGO_ENTIDAD.csv
Guardando diccionario_datos\Catálogos\CATALOGO_SI_NO.csv
Guardando diccionario_datos\Catálogos\CATALOGO_SEXO.csv


dict_keys(['CATALOGO_DICTAMEN', 'CATALOGO_ENTIDAD', 'CATALOGO_ESTATUS_CASO', 'CATALOGO_INSTITUCION', 'CATALOGO_MUNICIPIO', 'CATALOGO_RESULTADO_PCR', 'CATALOGO_SEXO', 'CATALOGO_SI_NO', 'CATALOGO_TIPO_PACIENTE'])

Cargando las tablas con datos del dengue del 2021 y 2022 en DF...

In [70]:
# Tablas de la DB: "dengue_301221" y "dengue_201222"...

# Seleccionar las variables a utilizar...
variables = ["FECHA_ACTUALIZACION", "SEXO", "EDAD_ANOS", "ENTIDAD_RES", "MUNICIPIO_RES", "FECHA_SIGN_SINTOMAS",
             "TIPO_PACIENTE", "HEMORRAGICOS", "DEFUNCION", "DICTAMEN", "RESULTADO_PCR", "ESTATUS_CASO"]
columnas_select = ', '.join(variables)
# Creando la Querry de forma segura...
query_21 = f"SELECT {columnas_select} FROM dengue_301221"
query_22 = f"SELECT {columnas_select} FROM dengue_201222"

df_dengue_2021 = pd.read_sql_query(query_21, connection, parse_dates= ["FECHA_ACTUALIZACION", "FECHA_SIGN_SINTOMAS"]) # Parser las fechas de una vez...
df_dengue_2022 = pd.read_sql_query(query_22, connection, parse_dates= ["FECHA_ACTUALIZACION", "FECHA_SIGN_SINTOMAS"])

# Mapear los catálogos para trabajar de forma cómoda con los datos...
df_dengue_2021["SEXO"] = df_dengue_2021["SEXO"].map(diccionarios["CATALOGO_SEXO"])
df_dengue_2021["ENTIDAD_RES"] = df_dengue_2021["ENTIDAD_RES"].map(diccionarios["CATALOGO_ENTIDAD"])
df_dengue_2021["MUNICIPIO_RES"] = df_dengue_2021["MUNICIPIO_RES"].map(diccionarios["CATALOGO_MUNICIPIO"])
df_dengue_2021["TIPO_PACIENTE"] = df_dengue_2021["TIPO_PACIENTE"].map(diccionarios["CATALOGO_TIPO_PACIENTE"])
df_dengue_2021["HEMORRAGICOS"] = df_dengue_2021["HEMORRAGICOS"].map(diccionarios["CATALOGO_SI_NO"])
df_dengue_2021["DEFUNCION"] = df_dengue_2021["DEFUNCION"].map(diccionarios["CATALOGO_SI_NO"])
df_dengue_2021["DICTAMEN"] = df_dengue_2021["DICTAMEN"].map(diccionarios["CATALOGO_DICTAMEN"])
df_dengue_2021["RESULTADO_PCR"] = df_dengue_2021["RESULTADO_PCR"].map(diccionarios["CATALOGO_RESULTADO_PCR"])
df_dengue_2021["ESTATUS_CASO"] = df_dengue_2021["ESTATUS_CASO"].map(diccionarios["CATALOGO_ESTATUS_CASO"])
df_dengue_2021["FECHA_SIGN_SINTOMAS"].sort_values()

df_dengue_2022["SEXO"] = df_dengue_2022["SEXO"].map(diccionarios["CATALOGO_SEXO"])
df_dengue_2022["ENTIDAD_RES"] = df_dengue_2022["ENTIDAD_RES"].map(diccionarios["CATALOGO_ENTIDAD"])
df_dengue_2022["MUNICIPIO_RES"] = df_dengue_2022["MUNICIPIO_RES"].map(diccionarios["CATALOGO_MUNICIPIO"])
df_dengue_2022["TIPO_PACIENTE"] = df_dengue_2022["TIPO_PACIENTE"].map(diccionarios["CATALOGO_TIPO_PACIENTE"])
df_dengue_2022["HEMORRAGICOS"] = df_dengue_2022["HEMORRAGICOS"].map(diccionarios["CATALOGO_SI_NO"])
df_dengue_2022["DEFUNCION"] = df_dengue_2022["DEFUNCION"].map(diccionarios["CATALOGO_SI_NO"])
df_dengue_2022["DICTAMEN"] = df_dengue_2022["DICTAMEN"].map(diccionarios["CATALOGO_DICTAMEN"])
df_dengue_2022["RESULTADO_PCR"] = df_dengue_2022["RESULTADO_PCR"].map(diccionarios["CATALOGO_RESULTADO_PCR"])
df_dengue_2022["ESTATUS_CASO"] = df_dengue_2022["ESTATUS_CASO"].map(diccionarios["CATALOGO_ESTATUS_CASO"])
df_dengue_2022["FECHA_SIGN_SINTOMAS"].sort_values()

df_dengue_2022.head()

,FECHA_ACTUALIZACION,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,FECHA_SIGN_SINTOMAS,TIPO_PACIENTE,HEMORRAGICOS,DEFUNCION,DICTAMEN,RESULTADO_PCR,ESTATUS_CASO
0,2022-12-12,MUJER,47,NAYARIT,JEREZ,2022-01-26,AMBULATORIO,NO,NO,NO APLICA,SIN SEROTIPO AISLADO,DESCARTADO
1,2022-12-12,MUJER,34,SINALOA,HUANUSCO,2022-01-29,AMBULATORIO,NO,NO,NO APLICA,SIN SEROTIPO AISLADO,DESCARTADO
2,2022-12-12,HOMBRE,50,QUINTANA ROO,APULCO,2022-01-29,AMBULATORIO,NO,NO,NO APLICA,SIN SEROTIPO AISLADO,DESCARTADO
3,2022-12-12,HOMBRE,23,VERACRUZ DE IGNACIO DE LA LLAVE,VERACRUZ,2022-01-28,HOSPITALIZADO,NO,NO,NO APLICA,SIN SEROTIPO AISLADO,DESCARTADO
4,2022-12-12,HOMBRE,23,GUERRERO,MIGUEL AUZA,2022-04-10,AMBULATORIO,NO,NO,NO APLICA,SIN SEROTIPO AISLADO,PROBABLE


In [71]:
# Listos entonces... a trabajar con los datos...



Casos confirmados para Dengue y Ch en Sonora:

In [12]:

df_confirmados_sonora = pd.read_sql_query("SELECT * \
                                            FROM vw_dengue_sonora \
                                            WHERE Dictamen = 1 \
                                                OR Dictamen = 2" \
                                        , connection)
df_confirmados_sonora



,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG


Realizando comparaciones entre las Bases de datos de los años 2021 y 2022...

,FECHA_ACTUALIZACION,ID_REGISTRO,SEXO,EDAD_ANOS,ENTIDAD_RES,MUNICIPIO_RES,HABLA_LENGUA_INDIG,INDIGENA,ENTIDAD_UM_NOTIF,MUNICIPIO_UM_NOTIF,...,INMUNOSUPR,CIRROSIS_HEPATICA,EMBARAZO,DEFUNCION,DICTAMEN,TOMA_MUESTRA,RESULTADO_PCR,ESTATUS_CASO,ENTIDAD_ASIG,MUNICIPIO_ASIG
0,2021-12-13,917221,2,10,25,6,2,2,25,6,...,2,2,2,2,5,1,1,2,25,6
1,2021-12-13,917230,1,19,25,6,2,2,25,6,...,2,2,2,2,5,2,5,1,25,6
2,2021-12-13,912536,2,14,7,13,2,2,7,101,...,2,2,2,2,5,1,5,3,7,13
3,2021-12-13,912584,1,48,17,11,2,2,17,7,...,2,2,2,2,5,2,5,1,17,11
4,2021-12-13,912581,2,22,14,98,2,2,14,39,...,2,2,2,2,5,2,5,1,14,98
5,2021-12-13,912587,1,31,14,101,2,2,14,101,...,2,2,2,2,5,1,5,3,14,101
6,2021-12-13,913082,1,54,21,141,2,2,21,3,...,2,2,2,2,5,1,5,3,21,141
7,2021-12-13,913085,1,11,5,2,2,2,5,2,...,2,2,2,2,5,1,1,2,5,2
8,2021-12-13,913111,2,57,30,44,2,2,30,44,...,2,2,2,2,5,1,2,2,30,44
9,2021-12-13,913108,1,53,6,1,2,2,6,1,...,2,2,2,2,5,1,5,3,6,1


In [14]:
df_dengue_2021.info()




<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35413 entries, 0 to 35412
Data columns (total 28 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   FECHA_ACTUALIZACION     35413 non-null  object 
 1   ID_REGISTRO             35413 non-null  int64  
 2   SEXO                    35413 non-null  int64  
 3   EDAD_ANOS               35413 non-null  int64  
 4   ENTIDAD_RES             35413 non-null  int64  
 5   MUNICIPIO_RES           35413 non-null  int64  
 6   HABLA_LENGUA_INDIG      35413 non-null  int64  
 7   INDIGENA                35413 non-null  int64  
 8   ENTIDAD_UM_NOTIF        35413 non-null  int64  
 9   MUNICIPIO_UM_NOTIF      35413 non-null  int64  
 10  INSTITUCION_UM_NOTIF    35413 non-null  int64  
 11  FECHA_SIGN_SINTOMAS     35413 non-null  object 
 12  TIPO_PACIENTE           35355 non-null  float64
 13  HEMORRAGICOS            35413 non-null  int64  
 14  DIABETES                35413 non-null

NameError: name 'FECHA_ACTUALIZACION' is not defined

### Cerrando la conexión a la BD

In [ ]:

"""
connection.commit()

# Cierra la conexión con la base de datos.
connection.close()
"""

In [18]:
diccionarios

{'DICTAMEN.csv': {1: 'CONFIRMADO A DENGUE',
  2: 'CONFIRMADO A CHIKUNGUNYA',
  3: 'NEGATIVO',
  4: 'EN ESTUDIO',
  5: 'NO APLICA'},
 'ENTIDAD.csv': {1: 'AGUASCALIENTES',
  2: 'BAJA CALIFORNIA',
  3: 'BAJA CALIFORNIA SUR',
  4: 'CAMPECHE',
  5: 'COAHUILA DE ZARAGOZA',
  6: 'COLIMA',
  7: 'CHIAPAS',
  8: 'CHIHUAHUA',
  9: 'CIUDAD DE MÉXICO',
  10: 'DURANGO',
  11: 'GUANAJUATO',
  12: 'GUERRERO',
  13: 'HIDALGO',
  14: 'JALISCO',
  15: 'MÉXICO',
  16: 'MICHOACÁN DE OCAMPO',
  17: 'MORELOS',
  18: 'NAYARIT',
  19: 'NUEVO LEÓN',
  20: 'OAXACA',
  21: 'PUEBLA',
  22: 'QUERÉTARO',
  23: 'QUINTANA ROO',
  24: 'SAN LUIS POTOSÍ',
  25: 'SINALOA',
  26: 'SONORA',
  27: 'TABASCO',
  28: 'TAMAULIPAS',
  29: 'TLAXCALA',
  30: 'VERACRUZ DE IGNACIO DE LA LLAVE',
  31: 'YUCATÁN',
  32: 'ZACATECAS',
  33: 'ESTADOS UNIDOS DE AMERICA',
  34: 'OTROS PAISES DE LATINOAMERICA',
  35: 'OTROS PAISES',
  97: 'NO APLICA',
  98: 'SE IGNORA',
  99: 'NO ESPECIFICADO'},
 'ESTATUS_CASO.csv': {1: 'PROBABLE', 2: 'CONFIR